In [ ]:
from indexing import *
from circuits import *
from expected import *
from grover import *
import qiskit as qk
import qiskit_aer
from qiskit.visualization import plot_histogram

def run_circuit(qc, shots=1000):
    backend = qiskit_aer.AerSimulator(method='statevector')
    compiled_circuit = qk.transpile(qc, backend)
    result = backend.run(compiled_circuit, shots=shots).result()
    counts = result.get_counts(compiled_circuit)
    return counts

def oracle(imat, pairs_known=None):
    ix, ds = dmat2ds(dmat_expected(imat))
    if pairs_known is None:
        ds_known = ds
    else:
        ix_known = set([ix.edge(*p) for p in pairs_known])
        ds_known = [(ix, d) for (ix, d) in ds if ix in ix_known]
    state_qubits = range(ix.num_edges)
    return Bitflip2Phaseflip(Dist_check(ix, ds_known)), state_qubits

def true_sol(imat):
    ix = ind.PathIndexing(imat.shape[0])
    return bools2str(imat2edges(ix, imat)[::-1])


In [ ]:
# Test with
# 0 - 1 - 2
imat = np.full((3,3), False)
imat[0, 1] = True
imat[1, 2] = True
# Pairs of nodes for which distances are known 
pairs_known = [(0,2)]

qc = grover(*oracle(imat, pairs_known))
counts = run_circuit(qc)
display(plot_histogram(counts))
print(f'True solution      {true_sol(imat)}')
print(f'Most frequent {counts.most_frequent()}')


In [ ]:
# Test with
# 0 - 1
#   / |
# 3 - 2
imat = np.full((4,4), False)
imat[0, 1] = True
imat[1, 2] = True
imat[1, 3] = True
imat[2, 3] = True
pairs_known = [(0,2), (0, 3), (2, 3)]

o, state = oracle(imat, pairs_known)
qc = grover(o, state)
counts = run_circuit(qc)
display(plot_histogram(counts))
print(f'True solution          {true_sol(imat)}')
print(f'Most frequent {counts.most_frequent()}')


In [ ]:
# Test Konigsberg
# 0 - 1 - 2
# | / | /
# 4 - 3
imat = np.full((5,5), False)
# Outer cycle
imat[0, 1] = True
imat[1, 2] = True
imat[2, 3] = True
imat[3, 4] = True
imat[0, 4] = True
# Inner edges
imat[1, 4] = True
imat[1, 3] = True

pairs_known = [(0,2), (0, 3), (0, 4), (2, 3), (2, 4), (3, 4)]
o, state = oracle(imat, pairs_known)
qc = grover(o, state, 1)
counts = run_circuit(qc, shots=1)
display(plot_histogram(counts))
print(f'True solution               {true_sol(imat)}')
print(f'Most frequent {counts.most_frequent()}')